In [32]:
import os
import networkx as nx
import pandas as pd
import numpy as np

# Load edges from .edges file
def load_edges(file_path):
    edges = []
    nodes = set()  # To track unique nodes
    with open(file_path, 'r') as f:
        for line in f:
            u, v = map(int, line.strip().split())
            edges.append((u, v))
            nodes.add(u)
            nodes.add(v)
    return edges, list(nodes)

# Load features from .feats and .egofeat files
def load_features(feat_path, egofeat_path, node_ids):
    # Read feature files
    features = pd.read_csv(feat_path, delim_whitespace=True, header=None)
    ego_features = pd.read_csv(egofeat_path, delim_whitespace=True, header=None)
    # Combine the features
    features = pd.concat([features, ego_features], ignore_index=True)
    
    # Check if number of features matches number of nodes
    if len(features) != len(node_ids):
        print(f"Warning: Number of features does not match number of nodes ({len(features)} vs {len(node_ids)})")
    
    # Create a mapping from node IDs to features
    node_feature_map = dict(zip(node_ids, features.values))
    return node_feature_map

# Build a graph from the dataset files
def build_graph(edges, node_feature_map):
    G = nx.DiGraph()
    G.add_edges_from(edges)
    
    # Add features as node attributes
    for node_id, features in node_feature_map.items():
        G.nodes[node_id]['feature'] = features.tolist()
    
    return G

# Process all networks in the dataset folder
def process_dataset(dataset_path):
    networks = {}
    
    # Check the dataset folder
    print("Files in dataset folder:")
    print(os.listdir(dataset_path))
    
    # Identify unique prefixes (e.g., 0, 1, ...)
    prefixes = set()
    for file_name in os.listdir(dataset_path):
        if file_name.endswith(".edges"):
            prefix = file_name.split('.')[0]
            prefixes.add(prefix)
    
    print("Identified prefixes for networks:")
    print(prefixes)
    
    # Process each prefix
    for prefix in sorted(prefixes):
        edges_path = os.path.join(dataset_path, f"{prefix}.edges")
        feat_path = os.path.join(dataset_path, f"{prefix}.feat")
        egofeat_path = os.path.join(dataset_path, f"{prefix}.egofeat")
        
        # Check if all required files exist
        if os.path.exists(edges_path) and os.path.exists(feat_path) and os.path.exists(egofeat_path):
            print(f"Processing network: {prefix}")
            edges, node_ids = load_edges(edges_path)
            node_feature_map = load_features(feat_path, egofeat_path, node_ids)
            graph = build_graph(edges, node_feature_map)
            networks[prefix] = graph
        else:
            print(f"Skipping {prefix}: Missing one or more files")
    
    return networks

# Path to the dataset folder
dataset_path = "dataset"

# Process all networks
networks = process_dataset(dataset_path)

# Check if any networks were processed
if not networks:
    print("No networks were processed. Please check the dataset structure and paths.")
else:
    # Access and analyze one of the networks
    example_prefix = list(networks.keys())[0]
    example_graph = networks[example_prefix]
    print(f"Network {example_prefix}:")
    print(f"  Nodes: {example_graph.number_of_nodes()}")
    print(f"  Edges: {example_graph.number_of_edges()}")


Files in dataset folder:
['0.circles', '0.edges', '0.egofeat', '0.feat', '0.featnames', '107.circles', '107.edges', '107.egofeat', '107.feat', '107.featnames', '1684.circles', '1684.edges', '1684.egofeat', '1684.feat', '1684.featnames', '1912.circles', '1912.edges', '1912.egofeat', '1912.feat', '1912.featnames', '3437.circles', '3437.edges', '3437.egofeat', '3437.feat', '3437.featnames', '348.circles', '348.edges', '348.egofeat', '348.feat', '348.featnames', '3980.circles', '3980.edges', '3980.egofeat', '3980.feat', '3980.featnames', '414.circles', '414.edges', '414.egofeat', '414.feat', '414.featnames', '686.circles', '686.edges', '686.egofeat', '686.feat', '686.featnames', '698.circles', '698.edges', '698.egofeat', '698.feat', '698.featnames']
Identified prefixes for networks:
{'0', '686', '1684', '107', '3980', '3437', '1912', '698', '414', '348'}
Processing network: 0
Processing network: 107
Processing network: 1684
Processing network: 1912
Processing network: 3437
Processing netwo

In [36]:
import networkx as nx
import numpy as np

# Constants for node states
UNINFECTED = 0
ACTIVE = 1
INFECTED = 2

# Independent Cascade model for rumor propagation
def independent_cascade(G, seed_nodes, activation_prob=0.1):
    # Initialize all nodes as Uninfected
    for node in G.nodes:
        G.nodes[node]['state'] = UNINFECTED
    
    # Seed nodes become Active
    for seed in seed_nodes:
        G.nodes[seed]['state'] = ACTIVE
    
    active_nodes = set(seed_nodes)
    newly_active = set(seed_nodes)
    
    # Simulation loop for rumor propagation
    while newly_active:
        next_active = set()
        for node in newly_active:
            # Propagate the rumor to neighbors
            for neighbor in G.neighbors(node):
                if G.nodes[neighbor]['state'] == UNINFECTED:  # Only uninfected nodes can be infected
                    # Propagate with a probability (activation)
                    if np.random.rand() < activation_prob:
                        next_active.add(neighbor)
                        G.nodes[neighbor]['state'] = ACTIVE  # Activate the node
        active_nodes.update(next_active)
        newly_active = next_active
    
    # Return the final set of infected nodes
    infected_nodes = [node for node, data in G.nodes(data=True) if data['state'] == INFECTED]
    return infected_nodes, active_nodes


# Example usage:
G = nx.erdos_renyi_graph(100, 0.05)  # Example graph, adjust size and density as needed
seed_nodes = [0, 1, 2]  # Example seed nodes to start the rumor

# Run the IC model
infected_nodes, active_nodes = independent_cascade(G, seed_nodes, activation_prob=0.1)
print(f"Final infected nodes: {infected_nodes}")
print(f"Final active nodes: {active_nodes}")


Final infected nodes: []
Final active nodes: {0, 1, 2, 69, 62}


In [37]:
def static_rumor_propagation(G, alpha=0.5):
    # Calculate probability of rumor spread using SRPM
    for u, v in G.edges:
        out_u = G.out_degree(u)  # OUT(u): Number of outgoing edges from u
        in_v = G.in_degree(v)    # IN(v): Number of incoming edges to v
        p_uv = (alpha * np.log(1 + out_u)) / (alpha * np.log(1 + out_u) + (1 - alpha) * np.log(1 + in_v))
        G[u][v]['p'] = p_uv  # Store the probability in the edge attribute
    
    return G


def dynamic_rumor_propagation(G, alpha_0=0.5, lambda_param=0.5, c1=0.1, c2=0.1, t_max=10):
    # Simulate the rumor spread dynamically over multiple time steps
    alpha_t = alpha_0
    for t in range(t_max):
        # Update probabilities at time t
        for u, v in G.edges:
            out_u = G.out_degree(u)
            in_v = G.in_degree(v)
            p_uv_t = (alpha_t * np.log(1 + out_u)) / (alpha_t * np.log(1 + out_u) + (1 - alpha_t) * np.log(1 + in_v))
            G[u][v]['p'] = p_uv_t
        
        # Calculate the dynamic popularity alpha_t
        A_t = [node for node, data in G.nodes(data=True) if data['state'] == ACTIVE]
        I_t = [node for node, data in G.nodes(data=True) if data['state'] == INFECTED]
        
        alpha_t = (lambda_param * len(A_t) + c1) / (len(I_t) + c2)
    
    return G


In [38]:
import random
import numpy as np

# Constants for node states
UNINFECTED = 0
ACTIVE = 1
INFECTED = 2
BLOCKED = 3

class RumorRL:
    def __init__(self, graph, activation_prob=0.1):
        self.G = graph
        self.activation_prob = activation_prob
        self.reset()

    # Reset the environment (state) to its initial configuration
    def reset(self):
        for node in self.G.nodes:
            self.G.nodes[node]['state'] = UNINFECTED
        self.state = [UNINFECTED] * len(self.G.nodes)
        return self.state
    
    # Simulate rumor spread and return the number of newly infected nodes
    def propagate_rumor(self, seed_nodes):
        active_nodes = set(seed_nodes)
        newly_active = set(seed_nodes)

        while newly_active:
            next_active = set()
            for node in newly_active:
                for neighbor in self.G.neighbors(node):
                    if self.G.nodes[neighbor]['state'] == UNINFECTED:
                        if np.random.rand() < self.activation_prob:
                            next_active.add(neighbor)
                            self.G.nodes[neighbor]['state'] = ACTIVE
            active_nodes.update(next_active)
            newly_active = next_active
        
        # Count the number of infected nodes
        infected_nodes = [node for node, data in self.G.nodes(data=True) if data['state'] == INFECTED]
        return len(infected_nodes)

    # Block a node by setting its state to BLOCKED
    def block_node(self, node_id):
        self.G.nodes[node_id]['state'] = BLOCKED
        self.state[node_id] = BLOCKED

    # Take an action (block a node) and return the new state and reward
    def step(self, action):
        # Block the selected node
        self.block_node(action)
        
        # Run the rumor propagation and calculate the reward
        infected_before = self.propagate_rumor([0])  # Example: assume rumor starts from node 0
        infected_after = self.propagate_rumor([0])  # After blocking one node
        reward = -(infected_after - infected_before)
        
        # Return the new state and the reward
        return self.state, reward


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from collections import deque

class QNetwork(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(QNetwork, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, output_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

class DQNAgent:
    def __init__(self, env, gamma=0.99, epsilon=1.0, epsilon_decay=0.995, min_epsilon=0.01, batch_size=32, memory_size=10000):
        self.env = env
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.min_epsilon = min_epsilon
        self.batch_size = batch_size
        self.memory = deque(maxlen=memory_size)
        
        self.input_dim = len(env.state)  # State dimension
        self.output_dim = len(env.G.nodes)  # Action space: Block any node
        self.q_network = QNetwork(self.input_dim, self.output_dim)
        self.target_network = QNetwork(self.input_dim, self.output_dim)
        self.target_network.load_state_dict(self.q_network.state_dict())
        self.optimizer = optim.Adam(self.q_network.parameters(), lr=0.001)
    
    def remember(self, state, action, reward, next_state):
        self.memory.append((state, action, reward, next_state))
    
    def act(self, state):
        if np.random.rand() < self.epsilon:
            return random.choice(range(self.output_dim))  # Random action (exploration)
        state_tensor = torch.FloatTensor(state).unsqueeze(0)
        q_values = self.q_network(state_tensor)
        return torch.argmax(q_values).item()  # Action with max Q-value (exploitation)
    
    def learn(self):
        if len(self.memory) < self.batch_size:
            return
        
        batch = random.sample(self.memory, self.batch_size)
        states, actions, rewards, next_states = zip(*batch)
        
        states_tensor = torch.FloatTensor(states)
        actions_tensor = torch.LongTensor(actions)
        rewards_tensor = torch.FloatTensor(rewards)
        next_states_tensor = torch.FloatTensor(next_states)
        
        # Get Q-values from the current network
        q_values = self.q_network(states_tensor).gather(1, actions_tensor.unsqueeze(1))
        
        # Get target Q-values from the target network
        next_q_values = self.target_network(next_states_tensor).max(1)[0].detach()
        target_q_values = rewards_tensor + self.gamma * next_q_values
        
        # Compute loss
        loss = nn.MSELoss()(q_values.squeeze(), target_q_values)
        
        # Optimize the Q-network
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        
        # Update epsilon for exploration-exploitation tradeoff
        if self.epsilon > self.min_epsilon:
            self.epsilon *= self.epsilon_decay
        
        # Periodically update the target network
        self.target_network.load_state_dict(self.q_network.state_dict())

# Example usage:
env = RumorRL(G)  # Assuming G is the graph created earlier
agent = DQNAgent(env)

# Training loop
for episode in range(1000):
    state = env.reset()
    total_reward = 0
    done = False
    
    while not done:
        action = agent.act(state)
        next_state, reward = env.step(action)
        agent.remember(state, action, reward, next_state)
        agent.learn()
        state = next_state
        total_reward += reward
    
    print(f"Episode {episode}: Total Reward = {total_reward}")
